In [5]:
#######################################################################
# MULTI-CLASS SVM USING SKLEARN
from sklearn import svm, datasets; import numpy as np
import matplotlib.pyplot as plt; import pandas as pd
import csv, sys

### Import data from CSV file
'''
rawdata = pd.read_csv('CuPcTsQCM20160811', delimiter='\t')
#to display all headers: list(rawdata.columns.values)
time = np.array(rawdata.elapsed_time_min)  # name array by its header in the data file
rawresponse = np.array(rawdata.srsQCM_ugCuPcTs)
pressure = np.array(rawdata.pressure_setpoint_Torr) #pressure (setpoint)
pressureact  = np.array(rawdata.pressure_Torr) #pressure (actual)
###
'''

var1 = np.array([1,2,3,4,9,9,10,8,15,16,14,17,25,26,28,27])
var2 = np.array([10,10,7,11,16,16,16,18,-2,-3,-4,-5,19,20,21,22])
classes = np.array([0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3])

h = .1 # step size in the mesh

#################
#insert arrays to be used as columns for input matrix (x and y coords.)
inputlist =(
    var1,
    var2)
#insert array to be used as column for target matrix (class number)
targetlist = (
    classes.astype(int))
  #  presscol)
##################

#see if input/target lists are multi-dimensional and convert to matrices
if len(np.shape(inputlist)) > 1:
    inputmatrix = np.array(zip(*inputlist)) #create input matrix
else:
    inputmatrix = inputlist.reshape(len(inputlist),) #change to '),1)' if necessary
    
if len(np.shape(targetlist)) > 1:
    targetmatrix = np.array(zip(*targetlist)) #create target matrix
else:
    targetmatrix = targetlist.reshape(len(targetlist),) #change to '),1)' if necessary

X = inputmatrix; y = targetmatrix


# create an instance of SVM and fit data
C = 1.0  # SVM regularization parameter
svc = svm.SVC(kernel='linear', C=C).fit(X, y)
rbf_svc = svm.SVC(kernel='rbf', gamma=0.7, C=C).fit(X, y)
poly_svc = svm.SVC(kernel='poly', degree=3, C=C).fit(X, y)
lin_svc = svm.LinearSVC(C=C).fit(X, y)

# create a mesh to plot in
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))

# title for the plots
titles = ['SVC with linear kernel',
          'LinearSVC (linear kernel)',
          'SVC with RBF kernel',
          'SVC with polynomial (degree 3) kernel']


for i, clf in enumerate((svc, lin_svc, rbf_svc, poly_svc)):
    # Plot the decision boundary. For that, we will assign a color to each
    # point in the mesh [x_min, x_max]x[y_min, y_max].
    plt.subplot(2, 2, i + 1)
    plt.subplots_adjust(wspace=0.4, hspace=0.4)

    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.contourf(xx, yy, Z, cmap=plt.cm.Paired, alpha=0.8)

    # Plot also the training points
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Paired)
    plt.xlabel('variable1')
    plt.ylabel('variable2')
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    #plt.xticks(())
    #plt.yticks(())
    plt.title(titles[i])

    # save all meshes
    meshname = 'multiclassmesh' + format(i) + '.csv'
    with open(meshname, "wb") as output:
        writer = csv.writer(output)
        writer.writerows(list(reversed(Z)))
    i += 1


plt.show()

#######################################################
# NEURAL NET REGRESSOR USING SKLEARN
'''
from sknn.mlp import Regressor, Layer
from sklearn.grid_search import GridSearchCV #auto network parameter optimization
from scipy import stats
from sklearn.grid_search import RandomizedSearchCV
from sklearn.cross_validation import train_test_split
import pickle #for saving trained networks


#design network
NNstarttime = timer() #start timer
nn = Regressor(layers=[
        Layer('Sigmoid', units=4), #hidden layers
        Layer('Sigmoid', units=3),
        Layer('Linear')], #output layer
    learning_rate=0.02,
    n_iter=100)

samplestart = 0 #set start index for samples
sampleend = 100 #set end index for samples
    

#create single columns for each input/output:
rescol = np.array([]); timecol = np.array([]); presscol = np.array([])

for i in range(0,len(ROIres),2):
    rescol = np.append(rescol, ROIres[i][samplestart:sampleend]) #response
#and times and pressures
    timecol = np.append(timecol, ROIt[0][samplestart:sampleend]) #time
    presscol = np.append(presscol, ROIpress[0][samplestart:sampleend]) #pressure
    
#single pressure column with all saturation (setpoint) pressure values
setpresscol = np.ones(len(rescol))
for i in range(len(abspress)):
    setpresscol[i*(sampleend-samplestart):i+2*i*(sampleend-samplestart)] = abspress[i]

#################
#insert data to be used as columns for input matrix 
inputlist =(
    rescol,
    timecol)
#insert data to be used as columns for target matrix
targetlist = (
    presscol)
  #  presscol)
##################

#see if input/target lists are multi-dimensional and convert to matrices

if len(np.shape(inputlist)) > 1:
    inputmatrix = np.array(zip(*inputlist)) #create input matrix
else:
    inputmatrix = inputlist.reshape(len(inputlist),1)
    
if len(np.shape(targetlist)) > 1:
    targetmatrix = np.array(zip(*targetlist)) #create target matrix
else:
    targetmatrix = targetlist.reshape(len(targetlist),1)

#split into random train and test sets
inp_train, inp_test, tar_train, tar_test = train_test_split(
    inputmatrix, targetmatrix, train_size=0.5, random_state = 0)

##################
nn.fit(inp_train, tar_train) #train network
prediction = nn.predict(inp_test) #make predictions
##################

# print(nn.get_parameters()) #summarize network parameters
NNendtime = timer()  #end training timer
NNtime = (NNendtime - NNstarttime) / 60 #total training time in minutes
print('NN training time (min):' + format(NNtime))
print('testing MSE (%): ' + format(MSE(tar_test,prediction)))

# print(nn.get_parameters()) #summarize network parameters

#input and target indexes to plot:
inpindex = 0 ; tarindex = 1

#see if input and target matrices are multi-dimensional
# to prevent plotting of an invalid matrix index
if len(np.shape(inputlist)) == 1: inpindex = 0
if len(np.shape(targetlist)) == 1: tarindex = 0

xvar = zip(*inp_test)[inpindex] #data to plot as "x" variable
yvar = zip(*tar_test)[tarindex] #data to plot as "y" variable
predictvar = zip(*prediction)[tarindex] #data to plot as predictions


plt.scatter(xvar, yvar, c='b', label='data') #test data
plt.scatter(xvar, predictvar, c='g', label='predictions') # predictions
plt.scatter(xvar, abs(np.subtract(yvar, predictvar)), c='r', label='errors') #errors
plt.legend(loc='lower right')
plt.xlabel('Input')
plt.ylabel('Target - actual and predicted')
plt.title('NN predictions and errors')
plt.show()

'''

#######################################################################
# SAVE NN RESULTS TO CSV FILE
'''
NNfile = "NNresults.csv"
saveNNdata = []; NNheaders = []; savetar = []

#see if input/target lists are multi-dimensional and reshape if necessary
#save test inputs:
if len(np.shape(inputlist)) > 1:
    for i in range(len(inputlist)):
        saveNNdata.append(zip(*inp_test)[i])
        NNheaders.append('test_input' + str(i+1))
else:
    saveNNdata.append(np.reshape(inp_test,len(inp_test)))
    NNheaders.append('test_input')
    
#save test targets and predictions:
if len(np.shape(targetlist)) > 1:
    for i in range(len(targetlist)):
        saveNNdata.append(zip(*tar_test)[i]) 
        NNheaders.append('test_target' + str(i+1))
        saveNNdata.append(zip(*prediction)[i])
        NNheaders.append('prediction' + str(i+1))
else:
    saveNNdata.append(np.reshape(tar_test,len(tar_test)))
    NNheaders.append('test_target')
    saveNNdata.append(np.reshape(prediction, len(tar_test)))
    NNheaders.append('prediction')
                        

    

saveNNdata = zip(*saveNNdata)
#open CSV file
with open(NNfile, "w") as output:
    writer = csv.writer(output, lineterminator='\n')
    writer.writerow(NNheaders) #write headers
    for row in saveNNdata:
        writer.writerow(row)
'''


#grid search optimizes network among the choices you give it:
'''
gs = GridSearchCV(nn, param_grid={
    'learning_rate': [0.05, 0.01, 0.005, 0.001],
    'hidden0__units': [4,8,12]
    'hidden0__type': ['Rectifier', 'Sigmoid', 'Tanh']})
gs.fit(X,y)
'''
# or among random parameters:
'''
gs = RandomizedSearchCV(nn, param_distributions={
    'learning_rate': stats.uniform[0.001, 0.1],
    'hidden0__units': stats.radint[3,15],
    'hidden0__type': ['Rectifier', 'Sigmoid', 'Tanh']})
gs.fit(X,y)
'''

#to save trained network:
#pickle.dump(nn, open('nn.pk1', 'wb'))

#to load trained network:
# nn = pickel.load(open('nn.pk1', 'rb'))



#######################################################################


ValueError: X.shape[1] = 2 should be equal to 3, the number of features at training time

In [6]:
X.shape

(16L, 3L)

In [3]:
X

array([[ 1, 10],
       [ 2, 10],
       [ 3,  7],
       [ 4, 11],
       [ 9, 16],
       [ 9, 16],
       [10, 16],
       [ 8, 18],
       [15, -2],
       [16, -3],
       [14, -4],
       [17, -5],
       [25, 19],
       [26, 20],
       [28, 21],
       [27, 22]])

In [4]:
y

array([0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3])